WHO-COVID-19-global-daily-data

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# define the model
def model(x, t, a, b, c):
    dxdt = a * x - b * x ** (1 + c)
    return dxdt

# parameters
a = 0.5
b = 0.2
c = 0.1

# initial condition
x0 = 1.0

# time points
t = np.linspace(0, 150, 1000)

# solve ODE
x = odeint(model, x0, t, args=(a, b, c))

# compute derivative of solution
dxdt = a * x - b * x ** (1 + c)

# create two subplots that share the same x-axis
fig, axs = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

# plot the solution on the first subplot
axs[0].plot(t, x)
axs[0].set_ylabel('x(t)')
axs[0].set_title('Solution of the differential equation')
axs[0].grid()

# plot the derivative on the second subplot
axs[1].plot(t, dxdt)
axs[1].set_xlabel('t')
axs[1].set_ylabel("x'(t)")
axs[1].set_title("Derivative of the solution")
axs[1].grid()

# layout so plots do not overlap
fig.tight_layout()

plt.show()

In [ ]:
import pandas as pd

df= pd.read_csv('WHO-COVID-19-global-daily-data.csv')


In [ ]:
start_date = '2020-02-15'
end_date = '2020-07-15'

italy_df = df[(df['Country'] == 'Italy') & 
              (df['Date_reported'] >= start_date) & 
              (df['Date_reported'] <= end_date)]

In [ ]:
# Assuming 'date' is in datetime format, you can plot the data as follows:
import matplotlib.dates as mdates


plt.figure(figsize=(12,6))
plt.plot(italy_df['Date_reported'], italy_df['New_cases'])  # Replace 'cases' with the actual column name
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('COVID-19 cases in Italy over time')
plt.grid()
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=30))

plt.show()

In [ ]:
from statsmodels.sandbox.regression.gmm import GMM

class gmm(GMM):
    def momcond(self, params):
        print('params=',params)
        alpha_1, alpha_2, epsilon = params
        N_t,N_t_1 = self.exog.T
        err = N_t - alpha_1*N_t_1 - alpha_2*N_t_1**(1+epsilon)
        return err


theta0 = np.array([1, 0.1, 0.3])

dta=pd.DataFrame(italy_df['New_cases'].fillna(0))
dta.insert(1, 'New_cases_lag_1', dta['New_cases'].shift(1).fillna(0))
dta.insert(2, 'New_cases_lag_2', dta['New_cases'].shift(2).fillna(0))
# dta.insert(0, 'Const', 1)

print(dta[10:20])
instrument_df = dta[['New_cases_lag_1', 'New_cases_lag_2']]
endog_df = dta[['New_cases', 'New_cases_lag_1']]
exog_df = endog_df
endog, exog, instrument  = map(np.asarray, [endog_df, exog_df, instrument_df])
endog1 = np.zeros(exog.shape[0])  
model = gmm(exog = exog, endog= endog1, instrument=instrument, k_moms=3, k_params=3)

model.fit(start_params=theta0,weights_method='hac')

In [ ]:
dta.to_stata('cases.dta')

